<table border="1" style="width:100%"><tr><td>
<h1 id="toctitle">Contents</h1>
<ul id="toc"/>
</td></tr></table>

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

* 이전 장(Ch.4)에서는 generalized linear model (GLM)에 대해서 논의했었다.
* GLM은 link function을 이용하여 설명 변수(explanatory variable) 간의 선형 조합과 반응 변수(response variable)을 연결시키는 것이었다.
* regression 문제를 풀기 위해 multiple linear regression 을 이용하는 방법을 배웠다.
* 그리고 classification 문제는 logistic regression을 이용하였다.

* 이번 장(Ch.5)에서는 classification, regression task 를 해결하는 방법으로 simple, non-linear model에 대해 이야기할 텐데,
* 이른바 decision tree 라는 것이다.
* 이 decision tree를 사용하여 웹 페이지의 이지미를 분석하고 광고인지 내용인지를 분류하는 ad blocker 를 만들어 볼꺼다.
* 마지막으로 ensemble learning methold에 대해 소개할 텐데, estimator를 조합하여 하나를 단독으로 쓰는 것보다 나은 성능을 내는 방법이다.

# Decision trees가 뭔가요?
* 결정 과정을 모델링하는 tree 비슷한 그래프. 스무고개("Twenty Questions")와 비슷하다.
* answerer (A)는 보통명사로 된 대상을 questioners (Qs)들에게 숨기고
* Qs은 yes, no, maybe 로 대답할 수 있는 질문을 최대 20회 던질 수 있다.
* 직관적인 전략은 specificity를 증가시키는 질문을 하는 것이다.
* 예를 들면 악기인가요? 같은 질문은 가능성을 효과적으로 줄여주지 못한다.
* decision tree 문제의 한 갈래로 response variable의 값을 추정하기 위하여 할 수 있는 explantory variable의 가장 작은 시퀀스를 찾는 문제가 있다.
* 어쨌든 아까의 비유를 계속하면 TQ에서는 Q / A 모두 training data에 대한 지식을 가지고 있다.
* 그러나 실제 문항에 대한 test instance에 대한 feature 값은 오직 A만이 알고 있다.

* Decision tree에서는 training instance 를 해당 instance의 explanatory variable 값에 따라
* 서브셋으로 나누는 과정을 재귀적(recursively)으로 진행함으로써 학습을 수행하게 된다.
* 아래 다이어그램이 전형적인 decision tree 의 예이다.
<img src="Mastering Machine Learning with scikit-learn.png">

* 각 box에는 explanatory variable을 테스트하는 수식이 들어있다.
* 테스트의 결과값과 edge로 연결된 노드들
* training instance는 각 test의 결과값에 따라 서브셋으로 나뉘게 된다.
* 테스트를 통과하면 오른쪽 child node로 가고, 통과하지 못하면 left child로 간다.
* 아래 child node에도 유사한 테스트들이 있고, stopping criterion이 만족될 때까지...
* classification task에서는 leaf node는 class를 나타낸다.
* regression task에서는 response variable의 estimate 하기 위해 각 leaf node에 담긴 instance의 response variable 을 평균하기도 한다.
* decision tree 를 만들고 나면, 테스트 instance에 대한 prediction은 leaf node에 도달할 때까지 위의 tree를 따라가기만 하면 된다.

# Training decision tree - 어떻게 decision tree 를 만들까?
역주) 위키피디아에 따르면 ID3 / C4.5 / C5.0 / CART / CHAID / MARS / Conditional Inference Tree 등 많은 decision tree 알고리즘이 있다.
* 여기서는 Iterative Dichotomiser 3 (ID3) 이라 불리는 알고리즘을 이용하여 decision tree를 만들어보자.


In [2]:
################# Training data #################
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report

instances = [
    {'plays fetch': True,  'Is grumpy': False, 'favoraite food': 'Bacon', 'species': 'Dog'},
    {'plays fetch': False, 'Is grumpy': True,  'favoraite food': 'Dog Food', 'species': 'Dog'},
    {'plays fetch': False, 'Is grumpy': True,  'favoraite food': 'Cat Food', 'species': 'Cat'},
    {'plays fetch': False, 'Is grumpy': True,  'favoraite food': 'Bacon', 'species': 'Cat'},
    {'plays fetch': False, 'Is grumpy': False, 'favoraite food': 'Cat Food', 'species': 'Cat'},
    {'plays fetch': False, 'Is grumpy': True,  'favoraite food': 'Bacon', 'species': 'Cat'},
    {'plays fetch': False, 'Is grumpy': True,  'favoraite food': 'Cat Food', 'species': 'Cat'},
    {'plays fetch': False, 'Is grumpy': False, 'favoraite food': 'Dog Food', 'species': 'Dog'},
    {'plays fetch': False, 'Is grumpy': True,  'favoraite food': 'Cat Food', 'species': 'Cat'},
    {'plays fetch': True,  'Is grumpy': False, 'favoraite food': 'Dog Food', 'species': 'Dog'},
    {'plays fetch': True,  'Is grumpy': False, 'favoraite food': 'Bacon', 'species': 'Dog'},
    {'plays fetch': False, 'Is grumpy': False, 'favoraite food': 'Cat Food', 'species': 'Cat'},
    {'plays fetch': True,  'Is grumpy': True,  'favoraite food': 'Cat Food', 'species': 'Cat'},
    {'plays fetch': True,  'Is grumpy': True,  'favoraite food': 'Bacon', 'species': 'Dog'}
]
df = pd.DataFrame(instances)

# print(df)
# df

<img src="training data.png">

데이터에서 알 수 있는 사실
* 개보다는 고양이가 까다롭다 혹은 까탈스럽(grumpier)다는 것을 알 수 있다.
    * 전체 개: 5      까다로운 개: 2
    * 전체 고양이: 8  까다로운 고양이: 6
* 대부분의 개는 "물어와 놀이(fetch game)"를 하지만 고양이는 잘 하지 않는다.
    * '물어와 놀이'를 하는 개: 4
    * '물어와 놀이'를 하는 고양이: 1
* 개들은 베이컨과 개사료를 비슷하게 좋아하는(2:3) 반면, 고양이는 베이컨 보다는 고양이 사료를 좋아한다(2:6)

* is grumpy, plays fetch 등의 explanatory variable은 쉽게 binary-valued variable로 바꿀 수 있다.
* favorite food 는 세 가지의 상태를 나타내는 categorical variable. one-hot ending 할 것임.
* 3장(Feature Extraction and Preprocessing)을 돌이켜보면, one-hot encoding 이라는 것은 categorical variable을 여러 개의 이진값으로 표시하는 것을 말한다.
* favorite food 는 세 개의 상태가 있기 때문에 세 개의 binary-valued feature로 나타낼 것이다.

이 테이블로부터 classification rule을 manually 구성할 수 있다.
* 예를 들면 까다롭(grumpy)고 고양이 사료(cat food)를 좋아하는 동물은 고양이 일것이고,
* 물어와 놀이를 하고, 베이컨을 좋아한다면 개일 것이다.
* 이렇게 manual classification rule을 구성하는 것은 적은 수의 데이터라도 귀찮고 성가신(cumbersome) 일이다.
* 그 대신 decision tree를 만들어서 이러한 rule을 학습하는 것을 배워보자.

## Selecting the questions - 질문 선택하기

* 여기서의 response variable 은 species임.
* 스무고개처럼 decistion tree는 exploratory variable의 값을 테스트함으로써 response variable의 값을 추정한다.
* 그럼 어떤 explanatory variable을 먼저 테스트 하면 좋을까?
* 직관적으로 생각해보면, 개나 고양이 만의 서브셋으로 분리해주는 것이 개와 고양이가 서로 섞인 - 잘 분리되지 않은 서브셋을 만드는 것보다 낫다.
* 서브셋에 포함된 각 멤버들이 서로 다른 클래스에 속한다면 해당 instance를 classify하기 어려울 것이다.
* 또한 고양이 한 마리, 개만 선별해내는 테스트도 피해야 할 것이다.
* 이러한 테스트는 스무고개에서 몇번 질문을 하지 않았는데도 구체적인 질문을 하는 것과 유사하다고 볼 수 있다.
* formally 포멀하게 말하자면 이러한 테스트들은 어쩌다가 한 두개의 instance를 분류할 뿐, 불확실성을 줄여주기 어려울 것이다.
* 분류(classification)에서의 불확실성을 가장 많이 줄이는 테스트가 가장 좋은 것이다.
* 불확실성의 양을 entropy라는 measure로 아래와 같이 수치화할 수 있다.
* 여기서 entropy는 bits로 측정되어지는데, 변수의 불확실성을 정량화할 수 있다.

$$ H ( X ) = - \sum_{i=1}^{n} P(x_i ) \log_b P(x_i ) $$

* $ n $: 구분할 수 있는 어떤 결과값의 총 갯수.
* $ x_i $: i번째 결과값.
* $ P(x_i )$: i번째 결과값 $x_i$ 가 나올 확률.

Example 1: a single coin toss 의 entropy
* 2 state: head / tail
* Prob = 1/2 = 0.5 (각각)
* $ H ( X ) = −(0.5 \log_2 0.5 + 0.5 \log_2 0.5) = 1.0 $
* 1 bit 로 기술 가능.

Example 2: two coin toss 의 entropy
* 4 state: head and head / head and tail / tail and head / tail and tail
* Prob = 1/2 x 1/2 = 0.25 (각각)
* $ H ( X ) = −(0.25\log_2 0.25 + 0.25\log_2 0.25 + 0.25\log_2 0.25 + 0.25\log_2 0.25) = 2.0 $
* 2 bit 로 기술 가능.

Example 3: coin의 양면이 head로 동일한 경우의 entropy
* 1 state: head
* Prob = 1
* $ H ( X ) = −(1 \log_2 1) = 0 $
* 0 bit / No information / 불확실성이 사라진 상태

Example 4: unfair coin의 entropy
* 1 state: head / tail
* Prob = 0.8 / 0.2
* $ H ( X ) = −(0.8 \log_2 0.8 + 0.2 \log_2 0.2) = 0.72192809488736231 $
* 0.7 bit 로 기술 가능.
* 두가지 가능성이 있지만, 어느 한쪽이 더 나오는 경향이 있다면 완전히 불확실한 것은 아니라는 식의 해석.

이제 어떤 species인지 모르는 동물을 classifying 하는 entropy를 계산해보자.
* 만일 training data에서 고양이와 개 비율이 동일하고 다른 정보는 없다면: entropy는 1 (Example 1)
* 그러나 training data를 보면...
    * dogs: 6 
    * cats: 8
* 따라서 entropy $ H(X) = - (\frac{6}{14} \log_2 \frac{6}{14} + \frac{8}{14} \log_2 \frac{8}{14}) = 0.985228136.342516 $
* cat 이 조금 더 많으므로 결과의 불확실성이 약간 낮은 것이다 라고 할 수 있겠다.

* 이제 entropy를 가장 많이 낮춰주는 explanatory variable 찾아보자.
* 시험삼아 play fetch 변수를 선택하고, 위의 training instance를 나눠보자.

In [3]:
################# Figures 05_01 #################
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report

instances = [
    {'plays fetch': True, 'species': 'Dog'},
    {'plays fetch': False, 'species': 'Dog'},
    {'plays fetch': False, 'species': 'Cat'},
    {'plays fetch': False, 'species': 'Cat'},
    {'plays fetch': False, 'species': 'Cat'},
    {'plays fetch': False, 'species': 'Cat'},
    {'plays fetch': False, 'species': 'Cat'},
    {'plays fetch': False, 'species': 'Dog'},
    {'plays fetch': False, 'species': 'Cat'},
    {'plays fetch': True, 'species': 'Dog'},
    {'plays fetch': True, 'species': 'Dog'},
    {'plays fetch': False, 'species': 'Cat'},
    {'plays fetch': True, 'species': 'Cat'},
    {'plays fetch': True, 'species': 'Dog'}
]
df = pd.DataFrame(instances)

X_train = [[1] if a else [0] for a in df['plays fetch']]
y_train = [1 if d == 'Dog' else 0 for d in df['species']]
labels = ['plays fetch']

clf = DecisionTreeClassifier(max_depth=None, max_features=None, criterion='entropy',
                             min_samples_leaf=1, min_samples_split=2)
print X_train
clf.fit(X_train, y_train)

[[1], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [0], [1], [1]]


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [4]:
f = 'tree.dot'
export_graphviz(clf, out_file=f, feature_names=labels, max_depth=None)

In [5]:
! dot -Tpng tree.dot -o tree.png

<img src="tree.png"/>

다이어그램의 해석
* 맨 위의 박스(root node): training instance 에 대해 테스트하는 explanatory variable이 무엇인지 알려주고 있다.
    * play fetch 를 테스트하고 있음
    * Boolean type (True / False)에서 binary-value (1 / 0) 으로 바꾸었음 (코드 참고)
    * Dog = 1, cat = 0 (코드 참고)
    * 아직 아무런 처리를 한 것이 아니므로 entropy 는 아까 계산한 0.985
    * (주) 테스트를 통과하면 왼쪽, 실패하면 오른쪽 child node로 간다.
* value는 cat=0 / dog=1의 순서
* play fetch = 0 인 training instance들은 테스트(play fetch < = 0.5)를 통과하므로 왼쪽의 child node로 간다
    * 이 서브셋에서는 cat : dog 의 비율이 7:2 이므로
    * entorpy $ H(X) = - (\frac{2}{9} \log_2 \frac{2}{9} + \frac{7}{9} \log_2 \frac{7}{9}) = 0.7642045065086203 $
* play fetch = 1 인 training instance들은 테스트(play fetch < = 0.5)를 통과하지 못하므로 오른쪽의 child node로 간다
    * 이 서브셋에서는 cat : dog 의 비율이 1:4 이므로
    * entorpy $ H(X) = - (\frac{1}{5} \log_2 \frac{1}{5} + \frac{4}{5} \log_2 \frac{4}{5}) = 0.7219280948873623 $

* 그럼 이번에는 'is grumphy'를 explanatory variable로 두고 테스트

In [6]:
################# Figures 05_02 #################
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report

instances = [
    {'is grumpy': False, 'species': 'Dog'},
    {'is grumpy': True, 'species': 'Dog'},ㄷ
    {'is grumpy': True, 'species': 'Cat'},
    {'is grumpy': True, 'species': 'Cat'},
    {'is grumpy': False, 'species': 'Cat'},
    {'is grumpy': True, 'species': 'Cat'},
    {'is grumpy': True, 'species': 'Cat'},
    {'is grumpy': False, 'species': 'Dog'},
    {'is grumpy': True, 'species': 'Cat'},
    {'is grumpy': False, 'species': 'Dog'},
    {'is grumpy': False, 'species': 'Dog'},
    {'is grumpy': False, 'species': 'Cat'},
    {'is grumpy': True, 'species': 'Cat'},
    {'is grumpy': True, 'species': 'Dog'}
]
df = pd.DataFrame(instances)
X_train = [[1] if a else [0] for a in df['is grumpy']]
y_train = [1 if d == 'Dog' else 0 for d in df['species']]
labels = ['is grumpy']
clf = DecisionTreeClassifier(max_depth=None, max_features=None, criterion='entropy',
                             min_samples_leaf=1, min_samples_split=2)
clf.fit(X_train, y_train)

f = 'is-grumpy.dot'
export_graphviz(clf, out_file=f, feature_names=labels)
! dot -Tpng is-grumpy.dot -o is-grumpy.png

SyntaxError: invalid syntax (<ipython-input-6-8106a8393627>, line 9)

<img src="is-grumpy.png"/>

* 마찬가지로 이번에는 favorite food 를 test variable로 두고 분석
* favoraite food = 'cat food' 인 경우에만 1, 나머지는 0

In [7]:
################# Figures 05_03 #################
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report

instances = [
    {'favorite food': False, 'species': 'Dog'},
    {'favorite food': False, 'species': 'Dog'},
    {'favorite food': True, 'species': 'Cat'},
    {'favorite food': False, 'species': 'Cat'},
    {'favorite food': True, 'species': 'Cat'},
    {'favorite food': False, 'species': 'Cat'},
    {'favorite food': True, 'species': 'Cat'},
    {'favorite food': False, 'species': 'Dog'},
    {'favorite food': True, 'species': 'Cat'},
    {'favorite food': False, 'species': 'Dog'},
    {'favorite food': False, 'species': 'Dog'},
    {'favorite food': True, 'species': 'Cat'},
    {'favorite food': True, 'species': 'Cat'},
    {'favorite food': False, 'species': 'Dog'}
]
df = pd.DataFrame(instances)
X_train = df[['favorite food']]

vectorizer = DictVectorizer()
X_train = [[1] if a else [0] for a in df['favorite food']]
y_train = [1 if d == 'Dog' else 0 for d in df['species']]

labels = ['favorite food=cat food']

clf = DecisionTreeClassifier(max_depth=None, max_features=None, criterion='entropy',
                             min_samples_leaf=1, min_samples_split=2)
print X_train
clf.fit(X_train, y_train)

f = 'favorite-food-cat-food.dot'
export_graphviz(clf, out_file=f, feature_names=labels)
! dot -Tpng favorite-food-cat-food.dot -o favorite-food-cat-food.png

[[0], [0], [1], [0], [1], [0], [1], [0], [1], [0], [0], [1], [1], [0]]


<img src="favorite-food-cat-food.png"/>

##Information gain
* cat food 선호 테스트로 나눈 서브셋 중 하나 (오른쪽 child node)를 보자.
* cat food만을 좋아하는 동물(=1)은 위의 테스트를 실패하므로 오른쪽 node로 가고 cat : dog = 6:0으로 entropy = 0인 상태를 만든다.
* 다른 쪽 서브셋 노드는 cat : dog = 2:6 이고 entropy = 0.8113
* 그럼 이제 어떤 테스트가 entropy 혹은 uncertainty를 가장 많이 떨어뜨렸는지 측정할 수 있나?
* 해당 서브셋의 entropy를 평균하는 것도 entropy reduction을 측정하는 적절한 방법이 되겠다.
* 직관적으로 보면, 전체 training 데이터의 거의 절반을 분류해내므로 효과적인 것처럼 보인다.
* 그러나... 
* 평균적으로 가장 낮은 entropy 를 가진 서브셋을 만들어주는 테스트를 선택하는 것은...
* (최적이 아니고 차적인) 부최적 트리(suboptimal tree)를 만들어낼 수 있다.
* 예를 들어 cat : dog = 0:2 과 8:4 인 두 서브셋을 만들어주는 테스트를 생각해보자.
    * 첫번째 서브셋의 entropy는 0 (한 가지 경우 밖에 없으므로)
    * 두번째 서브셋의 entropy $ H(X) = - (\frac{8}{12} \log_2 \frac{8}{12} + \frac{4}{12} \log_2 \frac{4}{12}) = 0.9182958340544896 $
    * 두 서브셋의 average entropy를 취하면 0.459 로 꽤 작지만, 대부분의 training instance가 포함된 두번째 서브셋의 entorpy는 거의 1에 가깝다.
    * 이것은 스무고개에서 많이 물어보지도 않고 구체적인 질문을 하는 경우와 유사하다.
    * 운이 좋으면 몇 번쯤 맞출 수도 있겠지만, 충분히 다른 가능한 경우를 제거할 수도 있는데 남아있는 질문들을 쓰지도 못하게 되는 셈이다.
* 이 방법 대신 information gain이라는 measure로 entropy reduction을 측정할 것이다.
    * parent node와의 entropy difference를 weighted average 한다. (weight은 child node에서의 entropy)

$$ IG (T,a) = H(T) - \sum_{\nu \in vals(a)} \frac{\{x \in T | x_a = \nu\}}{|T|} H(\{x \in T | x_a = \nu\}) $$

* 설명은 생략. 암튼 이것을 바탕으로 계산하면...

<img src="information gain1.png">

* cat food test가 가장 IG 가 높긴 하다.
* (주) 두 child 서브셋 중 하나의 entropy가 0 인 경우, IG는 나머지 다른 하나의 entropy의 절반의 값임. (weight = 0 이라서)

* 이제 tree 에 다른 노드도 더 해보자.
* cat food 테스트를 한 이후 entropy > 0 인 child 노드에 대해서 tree 확장을 해보자. cat : dog =  2:6
* 이 경우 역시 여러 테스트들을 해보고 IG 값을 비교해보면...
<img src="information gain2.png">

* child 노드의 entorpy = 0 을 주는 테스트 중 play fetch 와 is grumpy test 의 경우가 가장 IG 가 높으면서 같다. (tie)
* ID3 알고리즘은 임의로 둘 중 하나의 테스트를 선택하도록 되어있다.
* 여기서는 grumpy test 를 선택하는 것으로 하자.
* grumpy test 를 선택한 후 남은 non-zero entropy 서브셋에 대한 테스트를 하고 IG 값을 비교하여 (임의로) play fetch test를 선택한다.

<img src="information gain3.png"/>

<img src="decision tree level 2.png"/>

* 이제 슬슬 지겨워지는데... 어쨌든 다 했다고 보면..
<img src="decision tree level 4.png"/>

* 아래 test data를 위의 decision tree에 넣고 classify 해보자.
<img src="test data.png"/>

* instance 1
    * Favorite food = Bacon
    * Is grumpy = No
    * -> Dog
* instance 3
    * Favorite food = Dog food
    * Is grumpy = Yes
    * Plays fetch = No
    * Favorite food = Dog food
    * -> Dog (wrong!!)    

Other algorithms
* C4.5: a modified version of ID3 that can be used with continuous explanatory variables
    * and can accommodate missing values for features.
    * C4.5 also can prune trees. Pruning reduces the size of a tree by replacing branches
    * that classify few instances with leaf nodes.
    * Used by scikit-learn's implementation of decision trees,
* CART is another learning algorithm that supports pruning.

## Gini imputy
* 여기까지 최대의 IG (Information Gain)을 얻는 방법에 대해 배웠다.
* heuristic 한 다른 방법 중 하나는 Gini impurity 라는 방법이다.
    * 각 셋에 포함된 클래스의 비율을 측정하여 
$$ Gini(t) = 1 - \sum_{i=1}^j P(i|t)^2$$

* $ j $: 클래스의 수
* $ t $: 해당 노드에서의 서브셋 인덱스
* $ P(i|t) $: 그 서브셋에서 클래스 i 에 속하는 요소를 선택할 확률

* 동일한 클래스로 이루어진 서브셋의 경우: Gini = 0
* entropy 와 마찬가지로 여러 클래스에 속한 요소를 선택할 확률이 같을 때 최대값을 가진다.
* 주어진 클래스에 대해 최대값을 계산하는 공식은
$$ Gini_{\max} = 1 - \frac{1}{n}$$

* $ n $: 주어진 클래스의 수
* $ n=2 $ 인 경우, $ Gini_{\max} = 1 - 1/2 = 0.5 $

* scikit-learn에서는 information gain 및 Gini impurity 모두 지원한다.
* 이러한 measure는 반드시 한 가지로 고정되는 것도 아니며, 실제 분석할 때 보면 거의 비슷한 값을 준다.
* Machine Learning에서는 여러 옵션으로 선택하여 트레이닝된 모델의 성능을 비교하는 것이 가장 좋겠다.

# Decision trees with scikit-learn
* decision tree를 이용하여 ad blocker 를 만들어보자
* 이 프로그램에서는 웹페이지의 이미지가 내용을 담고 있는 것인지, 아니면 광고인지를 예측한다.
    * 광고라고 판단된 것은 CSS 를 이용하여 보이지 않게 처리할 수 있다.
* Internet Advertisements Data Set from http://archive.ics.uci.edu/ml/datasets/Internet+Advertisements
    * 3,279 장의 이미지인데 약간 편향되어 있다.
    * 459 장은 광고, 나머지 2,820은 광고가 아니다.
* 이렇게 클래스간의 비율이 unbalanced 된 데이터로 decision tree 알고리즘을 사용하면 편향된 트리(biased tree)를 만들어내게 된다.
* over- 혹은 under-sampling 을 이용하여 training 데이터의 balancing을 맞출 가치가 있다면,
* 결정하기 전에, 변화 전후의 데이터를 가지고 모델을 만들고 평가할 것이다. (?)

* explanatory variables
    * image 의 dimension / image를 포함한 page의 URL에서 추출한 단어 / image URL에서 추출한 단어
    * image alt text / anchor text / image tag 근처의 단어
* response variable: image class
    * advertisement or content
* 일단 expl. 들은 feature representatio 으로 변환.
* Data colums: width / height / aspect ratio / binary term frequencies (단어 요소들)
* 가장 큰 accurary를 주는 decision tree가 만들어지는 hyperparameter들에 대한 grid search를 할 예정.
* 그리고 test set 에 대한 각 tree의 성능 평가를 할 것임.

In [8]:
## data ##
! ls -l data/
! cat data/ad.DOCUMENTATION

total 10076
-rw-r--r-- 1 root root 10275015  7월  2 22:15 ad.data
-rw-r--r-- 1 root root     2103  7월  2 22:15 ad.DOCUMENTATION
-rw-r--r-- 1 root root    35579  7월  2 22:15 ad.names
1. Title of Database: Internet advertisements

2. Sources:
   (a) Creator & donor: Nicholas Kushmerick <nick@ucd.ie>
   (c) Generated: April-July 1998

3. Past Usage:
   N. Kushmerick (1999). "Learning to remove Internet advertisements",
   3rd Int Conf Autonomous Agents.  Available at
   www.cs.ucd.ie/staff/nick/research/download/kushmerick-aa99.ps.gz.
   Accuracy >97% using C4.5rules in predicting whether an image is an
   advertisement.

4. This dataset represents a set of possible advertisements on
   Internet pages.  The features encode the geometry of the image (if
   available) as well as phrases occuring in the URL, the image's URL and
   alt text, the anchor text, and words occuring near the anchor text.
   The task is to predict whether an image is an advertisement ("ad") or
   not ("nonad").

5. N

In [9]:
! head data/ad.data

 125, 125,   1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [10]:
!head data/ad.names

| "w:\c4.5\alladA" names file -- automatically generated!

ad, nonad | classes.

height: continuous.
width: continuous.
aratio: continuous.
local: 0,1.
| 457 features from url terms
url*images+buttons: 0,1.


* ad / contents 구분하는 것은 맨 마지막 컬럼에 있음.
* y 값은 ad의 경우 1, ad가 아니면 0
* white space 혹은 ? 로 되어 있는 missing value는 -1으로 할당하고
* imputation 할 때 평균값으로 대치한다고 함?
    * (주) pd.DataFrame.replace에서 inplace=True 로 설정하면 그렇게 된다는 이야기로 추정됨.
* train_test_split 을 이용하여 train data / test data 를 나눔.
* Pipeline을 만들고 DecisionTreeClassifier 를 할당한다.
* heuristic하게 parameter 값들의 range를 할당한다.
* model의 F1 score (F-score)를 (ref: https://en.wikipedia.org/wiki/F1_score) 최적화하는데에는 GridSearchCV() 라는 함수를 사용한다.
$$ F_1 = 2 \frac{\rm{precision} \rm{recall}}{\rm{precision} + \rm{recall}} $$

In [11]:
################# Sample 1: Ad classification with Decision Trees #################
import pandas as pd

help(pd.DataFrame.replace)

Help on method replace in module pandas.core.generic:

replace(self, to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad', axis=None) unbound pandas.core.frame.DataFrame method
    Replace values given in 'to_replace' with 'value'.
    
    Parameters
    ----------
    to_replace : str, regex, list, dict, Series, numeric, or None
    
        * str or regex:
    
            - str: string exactly matching `to_replace` will be replaced
              with `value`
            - regex: regexs matching `to_replace` will be replaced with
              `value`
    
        * list of str, regex, or numeric:
    
            - First, if `to_replace` and `value` are both lists, they
              **must** be the same length.
            - Second, if ``regex=True`` then all of the strings in **both**
              lists will be interpreted as regexs otherwise they will match
              directly. This doesn't matter much for `value` since there
              are on

In [12]:
################# Sample 1: Ad classification with Decision Trees #################
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

if __name__ == '__main__':
    df = pd.read_csv('data/ad.data', header=None)
    explanatory_variable_columns = set(df.columns.values)
    response_variable_column = df[len(df.columns.values)-1]
    # The last column describes the targets
    explanatory_variable_columns.remove(len(df.columns.values)-1)

    y = [1 if e == 'ad.' else 0 for e in response_variable_column]
    X = df[list(explanatory_variable_columns)]
    X.replace(to_replace=' *\?', value=-1, regex=True, inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    pipeline = Pipeline([
        ('clf', DecisionTreeClassifier(criterion='entropy'))
    ])
    parameters = {
        'clf__max_depth': (150, 155, 160),
        'clf__min_samples_split': (1, 2, 3),
        'clf__min_samples_leaf': (1, 2, 3)
    }

    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1')
    grid_search.fit(X_train, y_train)
    print 'Best score: %0.3f' % grid_search.best_score_
    print 'Best parameters set:'
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print '\t%s: %r' % (param_name, best_parameters[param_name])

    predictions = grid_search.predict(X_test)
    print classification_report(y_test, predictions)
    print grid_search.score(X_test, y_test)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    7.5s finished


Best score: 0.888
Best parameters set:
	clf__max_depth: 155
	clf__min_samples_leaf: 3
	clf__min_samples_split: 2
             precision    recall  f1-score   support

          0       0.97      0.99      0.98       712
          1       0.93      0.81      0.86       108

avg / total       0.97      0.97      0.96       820

0.861386138614


* 어쨌든 이렇게 돌려보니...
* Best score: 0.889
* Best parameters set:
	* clf__max_depth: 155
	* clf__min_samples_leaf: 1
	* clf__min_samples_split: 1
* test set에서 80% 이상의 광고를 걸러낸다.

* 수행 1번 에서
    * precision = 0.93: 광고라고 예측된 것은 93% 정도 진짜 광고였다.
    * recall = 0.81: hit rate (진짜 광고의 81%를 잡아내었다)

## Tree ensembles
* Ensemble learning: 여러 estimator를 조합하여 estimator를 만들어 각 estimator의 결과보다 너 나은 결과를 얻어내는 방법.
* Random Forest: Tree 가 여러 개 모여서 Forest가 되는 것임.
    * random 하게 선택된 training 데이터에 대해 수행한 decision tree의 collection
    * 일반적으로 mode 혹은 mean 결과를 내보낸다.
    * decision tree에 비해 overfitting 되지 않는다고 한다.
    * (주) overfitting tree 들의 영향이 다른 tree 들에 의해 average out 된다는 이야기
    * 노이즈에도 강하다고...

In [13]:
################# Sample 2: Ad classification with Random Forests #################
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

if __name__ == '__main__':
    df = pd.read_csv('data/ad.data', header=None)
    explanatory_variable_columns = set(df.columns.values)
    response_variable_column = df[len(df.columns.values)-1]
    # The last column describes the targets
    explanatory_variable_columns.remove(len(df.columns.values)-1)

    y = [1 if e == 'ad.' else 0 for e in response_variable_column]
    X = df[list(explanatory_variable_columns)]
    X.replace(to_replace=' *\?', value=-1, regex=True, inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    pipeline = Pipeline([
        ('clf', RandomForestClassifier(criterion='entropy'))
    ])
    parameters = {
        'clf__n_estimators': (5, 10, 20, 50),
        'clf__max_depth': (50, 150, 250),
        'clf__min_samples_split': (1, 2, 3),
        'clf__min_samples_leaf': (1, 2, 3)
    }

    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1')
    grid_search.fit(X_train, y_train)
    print 'Best score: %0.3f' % grid_search.best_score_
    print 'Best parameters set:'
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print '\t%s: %r' % (param_name, best_parameters[param_name])

    predictions = grid_search.predict(X_test)
    print classification_report(y_test, predictions)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 200 jobs       | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 310 out of 324 | elapsed:   30.8s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:   31.8s finished


Best score: 0.918
Best parameters set:
	clf__max_depth: 150
	clf__min_samples_leaf: 1
	clf__min_samples_split: 3
	clf__n_estimators: 20
             precision    recall  f1-score   support

          0       0.98      1.00      0.99       712
          1       1.00      0.90      0.95       108

avg / total       0.99      0.99      0.99       820



* 조금(?) 더 좋아졌다.
    * precision = 1.00 > 0.93
    * recall = 0.90 > 0.81
    * f1-score = 0.95 > 0.86
    
* Facebook에 조동환님이 올린 "Understanding Random Forests - From Theory to Practice"를 보면 좋을 것 같습니다.
    * https://www.facebook.com/groups/datakorea/478733745628775/
    * 필드에 있다보면 Random Forests 알고리즘이 왜 그런지 설명하기는 쉽지 않아도 가장 좋은 결과를 주는 경우가 "매우" 많습니다.
    * 제 생각에는 아마도 Bagging이라는 멋진 아이디어가 과적합(overfitting)도 막아내면서 예측력을 높이는 핵심이 되지 않나 싶습니다.
    * 첨부된 논문은 벨기에 리에주대학에서 작년에 박사학위 논문으로 제출한 "Understanding Random Forests - From Theory to Practice"입니다.
    * 이 논문에서는 왜 Random Forests 알고리즘에 잘 작동하는가에 대해서 이론적으로 설명하려는 시도를 하였습니다.

## The advantages and disdavantages of decision trees
* decision tree 는 쓰기 쉽다.
* normality (zero mean, unit variance) 가정 없이도 쓸 수 있다.
* missing value가 있어도 scikit-learn에서 imputation 해준다.

* 작은 크기의 decision tree 는 export_graphviz로 flowchart 처럼 그릴 수 있다.
* 큰 건 어렵다. F-score 같은 것으로 판단하는 게 좋을 듯.

* decision tree는 eager learner라고 한다.
    * 테스트 instance의 값을 예측하는데 쓰기 전에 미리 traingin 데이터로 모델을 만들어 두어야 한다.
    * 그러나 한번 만들어 두면 prediction은 매우 빠르다.
* 반면 k-nearest neighbor 와 같은 lazy learner 들이 있는데...
    * 예측을 해야 할 상황이 될 때까지 일반화를 미룬다고 한다.
    * 장점은 training 하느라 시간을 소비하지 않아도 되지만
    * 반면 예측하는데 시간이 많이 걸린다.
    
* Decision tree 는 over fitting 될 수 있다고 이야기하는데 이를 해결하기 위해 여러 방법이 동원된다.
    * Pruning 이라는 방법이 많이 쓰이는데, 가장 큰 (긴?) child 노드의 일부를 가위질 하는 것이다.
    * (scikit-learn에는 아직 구현되어 있지 않다.)
    * 그러나 tree의 maximum depth 를 조정하는 방식으로 비슷한 효과를 낼 수 있다.
* ID3와 같은 효율성이 좋은 decision tree 알고리즘은 greedy 하다고 함.
    * locally optimized decision을 하고 있으나
    * globally optimized decision을 보장해주지는 못한다.
    * 그래서 random forest를 쓰라고 하는 듯?
* 예제에서는 트리의 크기 - 노드의 수가 중요하지 않았는데...
    * 실제 예제에서는 pruning 혹은 다른 방법으로 tree 가 무한정 자라는 것을 제한한다.
    * information gain 혹은 Gini impurity 로 조절하는 locally optimized decision은 쓸만한 결과를 낸다고 함.

## Summary
* 위에 다 쓴 것 같네요...

# A tip to generate TOC
* [Sungjin Park: 각 입력 시퀀스 마다 링크가 있으면 좋겠네](https://www.facebook.com/oscarsjpark/posts/941976575845014)
  * [장혜식: TOC generator를 써봐요.](http://ludovicarnold.altervista.org/javascript-table-contents-ipython-notebook/)

\begin{align}
H(X) & = -\sum_{i=1}^{n} P(x_i) \log_b{P(x_i)} \\
\end{align}